In [1]:
using Pkg
Pkg.activate(@__DIR__)
pkg"precompile"

Precompiling project...


In [2]:
using MotionCaptureJointCalibration
using RigidBodyDynamics
using ValkyrieRobot
using Ipopt
using Random

Create a `Mechanism` (we'll use the NASA Valkyrie robot) and `MechanismState`:

In [3]:
val = Valkyrie()
mechanism = val.mechanism
remove_fixed_tree_joints!(mechanism)
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

Let's generate a random calibration problem, as well as its ground truth solution:

In [4]:
using MotionCaptureJointCalibration.SyntheticDataGeneration
bodies_with_markers = findbody.(Ref(mechanism), ["leftFoot", "pelvis"])
Random.seed!(1)
body_weights = Dict(b => rand() for b in bodies_with_markers)
problem, groundtruth = generate_calibration_problem(state, body_weights);

In [5]:
problem

CalibrationProblem{Float64} with:
* 25 poses
* 6 calibration parameters
* 8 markers attached to 2 bodies



In [6]:
groundtruth

CalibrationResult{Float64}: Optimal, residual = 0.0. Calibration parameters:
leftAnklePitch: [-0.00972919]
leftKneePitch: [0.000393507]
leftHipYaw: [0.00473574]
leftHipRoll: [0.00907607]
leftHipPitch: [-0.00809629]
leftAnkleRoll: [-0.00393202]

Solve the calibration problem (without using the ground truth result):

In [7]:
solver = IpoptSolver(print_level = 0, max_iter = 10000, check_derivatives_for_naninf = "yes", tol = 1e-10)
result = solve(problem, solver)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



CalibrationResult{Float64}: Optimal, residual = 1.1148577242390953e-9. Calibration parameters:
leftAnklePitch: [-0.00981715]
leftKneePitch: [0.000381069]
leftHipYaw: [0.00472127]
leftHipRoll: [0.00908977]
leftHipPitch: [-0.00804041]
leftAnkleRoll: [-0.00404071]

And visualize the results:

In [8]:
using RigidBodyTreeInspector
using Interact
using DrakeVisualizer

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/292f6e6efeb6819ddb313f5cc13018054e00772b-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/292f6e6efeb6819ddb313f5cc13018054e00772b-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Warning: Package MotionCaptureJointCalibration does not have DrakeVisualizer in its dependencies:
│ - If you have MotionCaptureJointCalibration checked out for development and have
│   added DrakeVisualizer as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MotionCaptureJointCalibration
│ Loading DrakeVisualizer into MotionCaptureJointCalibration from project dependency, future warnings for MotionCaptureJointCalibration are suppressed.
└ @ nothing nothing:840


In [9]:
#NBSKIP
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));

In [10]:
vis = Visualizer()[:valkyrie]
geometry = visual_elements(mechanism, URDFVisuals(ValkyrieRobot.urdfpath(); package_path = [ValkyrieRobot.packagepath()]))
setgeometry!(vis, mechanism, geometry)
state = MechanismState(mechanism)

MechanismState{Float64, Float64, Float64, …}(…)

In [11]:
inspect!(state, vis, problem, result)

(div { style=Dict("display"=>"flex","flex-direction"=>"column") }
  (div { className="field" }
        (div { attributes=Dict{Any,Any}("data-bind"=>"foreach : options_js") className=:field }
      (div { className="field" }
        (input { attributes=Dict{String,Any}("name"=>Symbol("##361"),"type"=>"radio","data-bind"=>"checked : \$root.index, checkedValue: val, attr : {id : id}") className="is-checkradio " })
        (label { attributes=Dict("data-bind"=>"text : key, attr : {for : id}") }))))
  (div { className="field" }
        (div { attributes=Dict("class"=>"interact-flex-row") }
      (div { attributes=Dict("class"=>"interact-flex-row-left") }
        (label { className="interact " style=Dict{Any,Any}(:padding=>"5px 10px 0px 10px") }
          "Pose number"))
      (div { attributes=Dict("class"=>"interact-flex-row-center") }
        (input { max=25 min=1 attributes=Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal") step=1 className="slider slider is-fullwidth" style=Dict{Any,Any}() }))
      (div { attributes=Dict("class"=>"interact-flex-row-right") }
        (p { attributes=Dict("data-bind"=>"text: value") })))))